# Eine StateMachine (SM) mit PyTransitions Library
https://github.com/pytransitions/transitions?tab=readme-ov-file#table-of-contents

Wichtig zu Verstehen ist, dass diese Bibliothek anders als State Pattern von GoF designt ist. Pytransitions unterscheidet zwischen Model und Zustandautomaten.

## Z.B. Model Tür:



In [ ]:
# Model einer Tür
class Door:
    def __init__(self, name):
        self.name = name

    def on_open(self):
        print(f"{self.name} ist jetzt offen.")

    def on_close(self):
        print(f"{self.name} ist jetzt geschlossen.")

    def on_lock(self):
        print(f"{self.name} ist jetzt verschlossen.")
        
    def on_unlock(self):
        print(f"{self.name} ist jetzt entsperrt.")


## Zustandsautomat

In [ ]:
from transitions import Machine

class DoorStateMachine(Machine):
    def __init__(self, model):
        # Definition der Zustände
        states = ['geschlossen', 'offen', 'verschlossen']

        # Definition der Übergänge
        # before bezieht sich auf Transition! 
        transitions = [
            {'trigger': 'open', 'source': 'geschlossen', 'dest': 'offen', 'before': 'on_open'}, # Führe im Zustand on_open aus, bevor SM zu Z open wechselt 
            {'trigger': 'close', 'source': 'offen', 'dest': 'geschlossen', 'before': 'on_close'},
            {'trigger': 'lock', 'source': 'geschlossen', 'dest': 'verschlossen', 'before': 'on_lock'},
            {'trigger': 'unlock', 'source': 'verschlossen', 'dest': 'geschlossen', 'before': 'on_unlock'}
        ]

        # Initialisieren der Machine mit den Zuständen und Übergängen
        super().__init__(model=model, states=states, transitions=transitions, initial='geschlossen')


## Verknüpfung zw. Model und Zustandsautomaten

In [ ]:
# Erstellen eines Door-Objekts (Model)
door = Door(name="Haustür")

# Erstellen der Zustandsmaschine und Verknüpfung mit dem Model
door_machine = DoorStateMachine(model=door)

# Zustandsübergänge ausführen
door.open()   # Haustür ist jetzt offen.
door.close()  # Haustür ist jetzt geschlossen.
door.lock()   # Haustür ist jetzt verschlossen.
door.unlock() # Haustür ist jetzt entsperrt.

### Beispiel für ein verbotenen Übergang

In [ ]:
door.lock()
door.open() #==> MachineError: "Can't trigger event lock from state verschlossen!"

In [ ]:
door.unlock()
door.open()
door.lock() #==> MachineError: "Can't trigger event lock from state offen!"

## Asynchrone State Machine mit pytransitions und Asyncio Extension
Beachte aus [Frequently asked questions - pytransitions](https://github.com/pytransitions/transitions/blob/master/examples/Frequently%20asked%20questions.ipynb): I have several inter-dependent machines/models and experience deadlocks.
#### Übergabe eigener Parameter an die Transitions CallBacks
>You can pass any number of arguments you like to the trigger.
>
>There is one important limitation to this approach: every callback function triggered by the state transition must be able to handle all of the arguments. This may cause problems if the callbacks each expect somewhat different data.
>
>To get around this, Transitions supports an alternate method for sending data. If you set `send_event=True` at Machine initialization, all arguments to the triggers will be wrapped in an EventData instance and passed on to every callback. (The EventData object also maintains internal references to the source state, model, transition, machine, and trigger associated with the event, in case you need to access these for anything.)

### Beispiel aus Doku: Very asynchronous Dancing: 
https://github.com/pytransitions/transitions/blob/master/examples/Playground.ipynb

In [ ]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio

class Dancer:
    
    states = ['start', 'left_food_left', 'left', 'right_food_right']
    
    def __init__(self, name, beat):
        self.my_name = name
        self.my_beat = beat
        self.moves_done = 0
        
    async def on_enter_start(self):
        self.moves_done += 1
        
    async def wait(self):
        print(f'{self.my_name} stepped {self.state}')
        await asyncio.sleep(self.my_beat)

    async def dance(self):
        while self.moves_done < 5:
            await self.step() #Acthung, dieser State muss mit 
        
dancer1 = Dancer('Tick', 0.5)
dancer2 = Dancer('Tock', 0.55)

m = AsyncMachine(model=[dancer1, dancer2], states=Dancer.states, initial='start', after_state_change='wait')
m.add_ordered_transitions(trigger='step')

# it starts okay but becomes quite a mess
_ = await asyncio.gather(dancer1.dance(), dancer2.dance()) 

## Benuzte AsyncState explizit
Ziel ist die Fähigkeit das State Design Pattern nach GoF zu programmieren

Beachte Antwort von Alexander Neumann ([aleneum](https://github.com/aleneum)) auf  [SO](https://stackoverflow.com/questions/78853312/how-to-use-asyncstate-class-of-pytransitions-asyncio-extension-explicitly/78855445#78855445 )

In [ ]:
from transitions.extensions.asyncio import AsyncMachine, AsyncState, AsyncEventData
import asyncio
import nest_asyncio
nest_asyncio.apply()


class StartState(AsyncState):
    def __init__(self, name, *args, **kwargs) -> None:
        super().__init__(name, *args, **kwargs)
        self.add_callback('enter', self.on_enter_state)

    async def on_enter_state(self, eventdata: AsyncEventData):
        print("On_enter start state")
        eventdata.model.moves_done += 1


class Dancer:
    states = [StartState(name='start'), 'left_food_left', 'left', 'right_food_right']

    def __init__(self, name, beat):
        self.my_name = name
        self.my_beat = beat
        self.moves_done = 0

    async def wait(self, event_data: AsyncEventData):
        print(f'{self.my_name} stepped {self.state}')
        await asyncio.sleep(self.my_beat)

    async def dance(self):
        while self.moves_done < 5:
            await self.step()


dancer1 = Dancer('Tick', 1)
dancer2 = Dancer('Tock', 1.1)

m = AsyncMachine(model=[dancer1, dancer2], states=Dancer.states, initial='start', after_state_change='wait', send_event=True)
m.add_ordered_transitions(trigger='step')

async def main():
    await asyncio.gather(dancer1.dance(), dancer2.dance())

asyncio.run(main())

# Übung: Heizung

In [ ]:
from transitions.extensions.asyncio import AsyncMachine, AsyncState, AsyncEventData, AsyncEvent
import asyncio
import datetime

def convert_to_time_obj(time_str):
    return datetime.strptime(time_str, "%H:%M").time()

def is_within_on_time_ranges(current_time, on_times, off_times):
    current_time = convert_to_time_obj(current_time)
    
    for on_time_str, off_time_str in zip(on_times, off_times):
        on_time = convert_to_time_obj(on_time_str)
        off_time = convert_to_time_obj(off_time_str)

        if on_time <= off_time:
            if on_time <= current_time <= off_time:
                return True
        else:
            # Dieser Fall behandelt Zeiten, die über Mitternacht gehen, z.B. 23:00 - 02:00
            if current_time >= on_time or current_time <= off_time:
                return True
    return False


class Heizung:
    states = ['off','abkuehlen', 'heizen', 'error']
    transitions = [
            { 'trigger': 'einschalten', 'source': 'off', 'dest': 'abkuehlen' },
            { 'trigger': 'heizung_an', 'source': 'abkuehlen', 'dest': 'heizen' },
            { 'trigger': 'heizung_aus', 'source': 'heizen', 'dest': 'abkuehlen' },
            { 'trigger': 'ausschalten', 'source': '*', 'dest': 'off' },
            { 'trigger': 'kaputt', 'source': '*', 'dest': 'error', 'after':'cancel_state_task' }   
        ]
    
    def __init__(self, targetT = 40, raumT = 18, on_times=["05:00", "14:00"], off_times = ["07:00", "18:00"]) -> None:
        self.sm = AsyncMachine(model=self, states = self.states, initial = 'off', transitions = self.transitions, send_event=True)
        self.targetT = targetT
        self.raumT = raumT
        self.switchEvent = AsyncEvent
        self.on_times = on_times
        self.off_times = off_times
        self.kessel_T = raumT
        self.state_task = None  # Referenz auf die heizen oder Abkühl-Task

    async def cancel_state_task(self, event_data = None):
        if self.state_task is not None:
            print("Beende Aktuellen Task")
            self.state_task.cancel()
            try:
                await self.state_task
            except asyncio.CancelledError:
                print("Task wurde beendet.")

    async def on_enter_heizen(self, event_data: AsyncEventData):
        print("Ich schalte den Kessel EIN")
        await self.cancel_state_task()
        if self.state_task is None or self.state_task.done():
            self.state_task = asyncio.create_task(self.do_heizen())

    async def on_exit_heizen(self, event_data: AsyncEventData):
        await self.cancel_state_task()

    async def do_heizen(self):
        '''
            Soll in einem eigenen Task oder Eventschleife ausgeführt werden.
        '''
        while True:
            if self.kessel_T < self.targetT:
                self.kessel_T += 0.5
            print(f"Heize, Kesseltemperatur: {self.kessel_T}°C")
            await asyncio.sleep(1.0)

    async def on_enter_abkuehlen(self, event_data: AsyncEventData):
        print("Ich bin nun in Kessel AUS Zustand")
        if self.state_task is None or self.state_task.done():
                self.state_task = asyncio.create_task(self.do_abkuehlen())        

    async def do_abkuehlen(self):
        '''
            Soll in einem eigenen Task ausgeführt werden.
        '''
        while True:
            if self.kessel_T > self.raumT:
                self.kessel_T -= 0.1
            print(f"Kessel ist aus, Kesseltemperatur: {self.kessel_T}°C")
            await asyncio.sleep(1.0)

    async def zeit_checken(self, testtime: str):    
        self.heizung_an() if is_within_on_time_ranges(testtime, self.on_times, self.off_times) else self.heizung_aus()

    async def on_enter_off(self, event_data):
        print("Schalte alles aus")
        await self.cancel_state_task()

    async def on_enter_error(self, event_data):
        print("Oh nein: Störung, führe Selbstdiagnose durch! Versuche aus und einzuschalten!")

h = Heizung()

In [ ]:
await h.einschalten()

In [ ]:
await h.heizung_an()

In [ ]:
await h.ausschalten()

In [ ]:
await h.kaputt()

In [ ]:
class PVWatcher:
    states =['waiting', 'waiting_for_triggerID', 'waiting_for_PV_Update', 'timeout_error']
    tansitions= [
        { 'trigger': 'new_triggerID', 'source': 'waiting', 'dest': 'waiting_for_PV_Update' },
        { 'trigger': 'PV_update', 'source': 'waiting', 'dest': 'waiting_for_triggerID' },
        { 'trigger': 'new_triggerID', 'source': 'waiting_for_triggerID', 'dest': 'waiting' },
        { 'trigger': 'PV_update', 'source': 'waiting_for_PV_Update', 'dest': 'waiting' },
    ]
    def __init__(self, timeout_triggerID, timeout_PVupdate = 0) -> None:
        self.timeout_triggerID = timeout_triggerID
        self.timeout_PVupdate = timeout_PVupdate
        self.newTriggerEvent = asyncio.Event()
        self.newPVUpdade = asyncio.Event()
        self.sm=AsyncMachine(model = self, states=self.states, initial='waiting', transitions=self.transitions, send_event=True)

    async def wait_for_event(self,  event : asyncio.Event, name: str, time_out: int):
        print(f"warte auf {name} ")
        try:
            event = await asyncio.wait_for(event.wait(), time_out)
        except asyncio.TimeoutError:
            pass

    async def on_enter_waiting(self, eventdata):
        t_wait_for_PVUpdata = asyncio.create_task(self.wait_for_event(self.newPVUpdade, 'eventPVUpdate'))
        t_wait_for_newTriggerEvent = asyncio.create_task(self.wait_for_event(self.newTriggerEvent, 'eventTriggerID'))
        try:
            done, pending = asyncio.wait([t_wait_for_newTriggerEvent, t_wait_for_PVUpdata], timeout=self.timeout_triggerID)
            # for d in done change state
        except asyncio.TimeoutError:
            pass # timeout_error
            


